In [5]:
import pandas as pd
import plotly.express as px
import numpy as np
from haversine import haversine
import plotly.graph_objects as go
import sys
sys.path.append('../utils/')
from clean import Clean
clean = Clean()

In [10]:
df = pd.read_csv('../dataset/train.csv')

df1 = df.copy()

In [11]:
df2 = clean.clean_code(df1)
df2.head()

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,Time_Order_picked,Weatherconditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken(min)
0,0x4607,INDORES13DEL02,37,4.9,22.745049,75.892471,22.765049,75.912471,2022-03-19,11:30:00,11:45:00,conditions Sunny,High,2,Snack,motorcycle,0,No,Urban,24
1,0xb379,BANGRES18DEL02,34,4.5,12.913041,77.683237,13.043041,77.813237,2022-03-25,19:45:00,19:50:00,conditions Stormy,Jam,2,Snack,scooter,1,No,Metropolitian,33
2,0x5d6d,BANGRES19DEL01,23,4.4,12.914264,77.678400,12.924264,77.688400,2022-03-19,08:30:00,08:45:00,conditions Sandstorms,Low,0,Drinks,motorcycle,1,No,Urban,26
3,0x7a6a,COIMBRES13DEL02,38,4.7,11.003669,76.976494,11.053669,77.026494,2022-04-05,18:00:00,18:10:00,conditions Sunny,Medium,0,Buffet,motorcycle,1,No,Metropolitian,21
4,0x70a2,CHENRES12DEL01,32,4.6,12.972793,80.249982,13.012793,80.289982,2022-03-26,13:30:00,13:45:00,conditions Cloudy,High,1,Snack,scooter,1,No,Metropolitian,30


In [3]:
linhas_selcionadas = (df1['Delivery_person_Age'] != 'NaN ')
df1 = df1.loc[linhas_selcionadas, :].copy()
linhas_selcionadas = (df1['multiple_deliveries'] != 'NaN ')
df1 = df1.loc[linhas_selcionadas, :].copy()
linhas_selcionadas = (df1['Road_traffic_density'] != 'NaN ')
df1 = df1.loc[linhas_selcionadas, :].copy()
linhas_selcionadas = (df1['Time_taken(min)'] != 'NaN')
df1 = df1.loc[linhas_selcionadas, :].copy()
linhas_selcionadas = (df1['City'] != 'NaN ')
df1 = df1.loc[linhas_selcionadas, :].copy()
linhas_selcionadas = (df1['Festival'] != 'NaN ')
df1 = df1.loc[linhas_selcionadas, :].copy()

df1['Time_taken(min)'] = df1['Time_taken(min)'].str.strip()
df1.replace("NaN", np.nan, inplace=True)
df1.dropna(inplace=True)


df1['Time_taken(min)'] = df1['Time_taken(min)'].apply( lambda x: x.split('(min) ')[1])

df1['Time_taken(min)'] = df1['Time_taken(min)'].astype(int)

df1['Delivery_person_Age'] = df1['Delivery_person_Age'].astype( int )

df1['Delivery_person_Ratings'] = df1['Delivery_person_Ratings'].astype( float )

df1['Order_Date'] = pd.to_datetime( df1['Order_Date'], format='%d-%m-%Y' )

df1['multiple_deliveries'] = df1['multiple_deliveries'].astype( int )

df1 = df1.reset_index(drop=True)

#removendo os espaços dentro
#for i in range(0, len(df1)):
    #df1.loc[i, 'ID'] = df1.loc[i, 'ID'].strip()
df1.loc[ : , 'ID'] =  df1.loc[ : ,'ID'].str.strip()
df1.loc[ : , 'Road_traffic_density'] =  df1.loc[ : ,'Road_traffic_density'].str.strip()
df1.loc[ : , 'Type_of_order'] =  df1.loc[ : ,'Type_of_order'].str.strip()
df1.loc[ : , 'Type_of_vehicle'] =  df1.loc[ : ,'Type_of_vehicle'].str.strip()
df1.loc[ : , 'City'] =  df1.loc[ : ,'City'].str.strip()
df1.loc[ : , 'Festival'] =  df1.loc[ : ,'Festival'].str.strip()

In [8]:
df1['City'].sort_values().unique()


array(['Metropolitian', 'Semi-Urban', 'Urban'], dtype=object)

#### SAPE

1. SAída

O que você vai entrefar como solução?

2. Processo

Qual a sequencia de passos logicos que eu devo seguir?

3. Entrada

Quais são as ferramentas da sua caixa que mais ajudariam a resolver o problema?

### Agrupamento

In [5]:
cols = ['ID', 'City']

group = df1.loc[: , cols].groupby(['City']).count().reset_index()

In [6]:
group

,City,ID
0,Metropolitian,3105
1,Semi-Urban,15
2,Urban,914


In [7]:
cols = ['Delivery_person_Age','Type_of_vehicle','City']
age = df1.loc[: , cols].groupby(['City', 'Type_of_vehicle']).mean().reset_index()
age

,City,Type_of_vehicle,Delivery_person_Age
0,Metropolitian,electric_scooter,30.300885
1,Metropolitian,motorcycle,29.600954
2,Metropolitian,scooter,29.786290
3,Semi-Urban,motorcycle,34.000000
4,Semi-Urban,scooter,39.000000
5,Urban,electric_scooter,28.444444
6,Urban,motorcycle,28.644269
7,Urban,scooter,28.822630


1. Quantidade de pedidos por dia.

## Visão da Empresa

In [8]:
cols = ['ID','Order_Date']

df_aux = df1.loc[:,cols].groupby('Order_Date').count().reset_index()

df_aux.head()

,Order_Date,ID
0,2022-02-11,88
1,2022-02-12,84
2,2022-02-13,85
3,2022-02-14,60
4,2022-02-15,86


In [9]:
# Desenhar Graficos
px.bar(df_aux,'Order_Date','ID')

2. Quantidade de pedidos por semana

In [10]:
df1['week_of_year'] = df1['Order_Date'].dt.strftime( '%U')

In [11]:
df1.head()
df1['week_of_year'] = df1['Order_Date'].dt.strftime( '%U')

df_aux = df1.loc[: ,['ID', 'week_of_year']].groupby('week_of_year').count().reset_index()

px.line(df_aux,'week_of_year','ID')

3. Distribuição dos pedidos por tipo de tráfego.

In [12]:
df_aux = df1.loc[:,['ID','Road_traffic_density']].groupby('Road_traffic_density').count().reset_index()

df_aux['entregas_perc'] = df_aux['ID'] / df_aux['ID'].sum()

px.pie(df_aux, values='entregas_perc',names='Road_traffic_density')


4. Comparação do volume de pedidos por cidade e tipo de tráfego.

In [13]:
df_aux = df1.loc[: , ['ID','City', 'Road_traffic_density']].groupby(['City', 'Road_traffic_density']).count().reset_index()

px.scatter(df_aux, x='City', y='Road_traffic_density',size='ID',color='City')

5. A localização central de cada cidade por tipo de tráfego.5

In [14]:
df_aux01 = df1.loc[ : , ['ID','week_of_year']].groupby('week_of_year').count().reset_index()
df_aux02 = df1.loc[ :, ['Delivery_person_ID', 'week_of_year']].groupby('week_of_year').nunique().reset_index()

df_aux = pd.merge( df_aux01, df_aux02, how='inner')
df_aux['order_by_deliver'] = df_aux['ID'] / df_aux['Delivery_person_ID']

px.line(df_aux, x='week_of_year', y='order_by_deliver')

6. A localização central de cada cidade por tipo de
tráfego.

In [15]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4034 entries, 0 to 4033
Data columns (total 21 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   ID                           4034 non-null   object        
 1   Delivery_person_ID           4034 non-null   object        
 2   Delivery_person_Age          4034 non-null   int64         
 3   Delivery_person_Ratings      4028 non-null   float64       
 4   Restaurant_latitude          4034 non-null   float64       
 5   Restaurant_longitude         4034 non-null   float64       
 6   Delivery_location_latitude   4034 non-null   float64       
 7   Delivery_location_longitude  4034 non-null   float64       
 8   Order_Date                   4034 non-null   datetime64[ns]
 9   Time_Orderd                  4034 non-null   object        
 10  Time_Order_picked            4034 non-null   object        
 11  Weatherconditions            4034 non-null   obj

In [16]:
import folium

df_aux = df1.loc[:, ['City', 'Road_traffic_density', 'Delivery_location_latitude', 'Delivery_location_longitude']].groupby(['City', 'Road_traffic_density']).median().reset_index()
df_aux = df_aux.loc[df_aux['City'] != 'NaN', :]
df_aux = df_aux.loc[df_aux['Road_traffic_density'] != 'NaN', :]

map = folium.Map()
for index , location_info in df_aux.iterrows():
    folium.Marker([location_info.loc['Delivery_location_latitude',],location_info.loc['Delivery_location_longitude']],popup=location_info[['City','Road_traffic_density']]).add_to(map)

map

## Visão dos entregadores

1. A menor e maior idade dos entregadores.

In [17]:
df1.columns

Index(['ID', 'Delivery_person_ID', 'Delivery_person_Age',
       'Delivery_person_Ratings', 'Restaurant_latitude',
       'Restaurant_longitude', 'Delivery_location_latitude',
       'Delivery_location_longitude', 'Order_Date', 'Time_Orderd',
       'Time_Order_picked', 'Weatherconditions', 'Road_traffic_density',
       'Vehicle_condition', 'Type_of_order', 'Type_of_vehicle',
       'multiple_deliveries', 'Festival', 'City', 'Time_taken(min)',
       'week_of_year'],
      dtype='object')

In [18]:
max = df1.loc[:,'Delivery_person_Age'].max()
min = df1.loc[:,'Delivery_person_Age'].min()
print(f'A maior idade é {max}')
print(f'A menor idade é {min}')

A maior idade é 39
A menor idade é 20


2. A pior e a melhor condição de veículos.

In [19]:
max = df1.loc[:,'Vehicle_condition'].max()
min = df1.loc[:,'Vehicle_condition'].min()
print(f'A melhor condição de vieculo é {max}')
print(f'A pior condição de vieculo é {min}')

A melhor condição de vieculo é 2
A pior condição de vieculo é 0


3. A avaliação média por entregador

In [20]:
df_avg_ratings = df1.loc[: , ['Delivery_person_ID','Delivery_person_Ratings']].groupby('Delivery_person_ID').mean().reset_index()
df_avg_ratings

,Delivery_person_ID,Delivery_person_Ratings
0,AGRRES010DEL01,4.700000
1,AGRRES010DEL02,4.700000
2,AGRRES01DEL02,4.400000
3,AGRRES01DEL03,4.800000
4,AGRRES02DEL02,4.900000
...,...,...
1115,VADRES19DEL02,4.528571
1116,VADRES19DEL03,4.640000
1117,VADRES20DEL01,4.680000
1118,VADRES20DEL02,4.533333


4. A avaliação média e o desvio padrão por tipo de tráfego.

In [21]:
#df_avg_ratings = ( df1.loc[ : , ['Delivery_person_Ratings','Road_traffic_density']]
                  #.groupby('Road_traffic_density').mean().reset_index())

df_std_ratings = (df1.loc[ : , ['Delivery_person_Ratings','Road_traffic_density']]
                  .groupby('Road_traffic_density').agg({'Delivery_person_Ratings':['mean','std']}))
df_std_ratings.columns = ['delivery_mean','delivery_std']
df_std_ratings.reset_index()


,Road_traffic_density,delivery_mean,delivery_std
0,High,4.672346,0.260760
1,Jam,4.600402,0.315273
2,Low,4.633022,0.351357
3,Medium,4.648381,0.290176


5. A avaliação média e o desvio padrão por condições climáticas.

In [22]:
df_std_ratings = (df1.loc[ : , ['Delivery_person_Ratings','Weatherconditions']]
                  .groupby('Weatherconditions').agg({'Delivery_person_Ratings':['mean','std']}))

df_std_ratings.columns = ['delivery_mean','delivery_std']
df_std_ratings.reset_index()

,Weatherconditions,delivery_mean,delivery_std
0,conditions Cloudy,4.620240,0.290688
1,conditions Fog,4.653343,0.266458
2,conditions Sandstorms,4.633588,0.297546
3,conditions Stormy,4.608018,0.314613
4,conditions Sunny,4.646605,0.419414
5,conditions Windy,4.620913,0.303915


6. Os 10 entregadores mais rápidos por cidade.

In [23]:
df1.head()

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,...,Weatherconditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken(min),week_of_year
0,0x4607,INDORES13DEL02,37,4.9,22.745049,75.892471,22.765049,75.912471,2022-03-19,11:30:00,...,conditions Sunny,High,2,Snack,motorcycle,0,No,Urban,24,11
1,0xb379,BANGRES18DEL02,34,4.5,12.913041,77.683237,13.043041,77.813237,2022-03-25,19:45:00,...,conditions Stormy,Jam,2,Snack,scooter,1,No,Metropolitian,33,12
2,0x5d6d,BANGRES19DEL01,23,4.4,12.914264,77.678400,12.924264,77.688400,2022-03-19,08:30:00,...,conditions Sandstorms,Low,0,Drinks,motorcycle,1,No,Urban,26,11
3,0x7a6a,COIMBRES13DEL02,38,4.7,11.003669,76.976494,11.053669,77.026494,2022-04-05,18:00:00,...,conditions Sunny,Medium,0,Buffet,motorcycle,1,No,Metropolitian,21,14
4,0x70a2,CHENRES12DEL01,32,4.6,12.972793,80.249982,13.012793,80.289982,2022-03-26,13:30:00,...,conditions Cloudy,High,1,Snack,scooter,1,No,Metropolitian,30,12


In [24]:
#df1.loc[0, 'Time_taken(min)'].split(' ')[1]
#df1.loc[0, 'Time_taken(min)'].replace('(min) ', '')
#df1['Time_taken(min)'].value_counts()

df1['Time_taken(min)'] = df1['Time_taken(min)'].astype(int)

In [25]:

df1['Time_taken(min)'] = df1['Time_taken(min)'].astype(int)
df2 = (df1.loc[:, ['Delivery_person_ID','City','Time_taken(min)']]
       .groupby(['Delivery_person_ID','City'])
       .min()
       .sort_values(['City','Time_taken(min)']).reset_index())

dfa1 =df2.loc[df2['City'] == 'Metropolitian', :].head(10)
dfa2 = df2.loc[df2['City'] == 'Urban', :].head(10)
dfa3 = df2.loc[df2['City'] == 'Semi-Urban', :].head(10)

pd.concat([dfa1,dfa2,dfa3]).reset_index(drop=True)

,Delivery_person_ID,City,Time_taken(min)
0,ALHRES08DEL03,Metropolitian,10
1,ALHRES19DEL03,Metropolitian,10
2,CHENRES04DEL03,Metropolitian,10
3,CHENRES20DEL02,Metropolitian,10
4,GOARES14DEL01,Metropolitian,10
5,HYDRES01DEL02,Metropolitian,10
6,HYDRES09DEL02,Metropolitian,10
7,HYDRES13DEL02,Metropolitian,10
8,HYDRES13DEL03,Metropolitian,10
9,INDORES19DEL01,Metropolitian,10


7. Os 10 entregadores mais lentos por cidade.

In [26]:
df2 = (df1.loc[:, ['Delivery_person_ID','City','Time_taken(min)']]
       .groupby(['Delivery_person_ID','City'])
       .max()
       .sort_values(['City','Time_taken(min)'], ascending=False)
       .reset_index())

dfa1 =df2.loc[df2['City'] == 'Metropolitian', :].head(10)
dfa2 = df2.loc[df2['City'] == 'Urban', :].head(10)
dfa3 = df2.loc[df2['City'] == 'Semi-Urban', :].head(10)

pd.concat([dfa1,dfa2,dfa3]).reset_index(drop=True)

,Delivery_person_ID,City,Time_taken(min)
0,CHENRES04DEL01,Metropolitian,54
1,LUDHRES18DEL03,Metropolitian,54
2,MYSRES11DEL03,Metropolitian,54
3,AGRRES03DEL01,Metropolitian,53
4,HYDRES06DEL01,Metropolitian,53
5,MYSRES010DEL01,Metropolitian,53
6,VADRES16DEL03,Metropolitian,53
7,BANGRES03DEL02,Metropolitian,52
8,BANGRES08DEL02,Metropolitian,52
9,COIMBRES06DEL01,Metropolitian,52


## Visão dos Restaurantes

1. A quantidade de entregadores únicos.

In [27]:
delivery_unique = len( df1.loc[:, 'Delivery_person_ID'].unique())
delivery_unique

print(f'A quantidade e entregadores Unicos é: {delivery_unique}')

A quantidade e entregadores Unicos é: 1120


2. A distância média dos resturantes e dos locais de entrega.

In [28]:
cols = ['Delivery_location_latitude', 'Delivery_location_longitude','Restaurant_latitude', 'Restaurant_longitude']

lat1 ='Delivery_location_latitude'
long1 ='Delivery_location_longitude'
lat2 ='Restaurant_latitude'
long2 ='Restaurant_longitude'

df1['distance'] = df1.loc[:, cols].apply(lambda x: haversine((x[lat1], x[long1]), (x[lat2], x[long2])), axis=1)
avg_distance = df1['distance'].mean()

print(f'A distiancia  media das entregas é: {avg_distance:.2f} km')

A distiancia  media das entregas é: 28.50 km


In [15]:
cols = ['Delivery_location_latitude', 'Delivery_location_longitude','Restaurant_latitude', 'Restaurant_longitude']

lat1 ='Delivery_location_latitude'
long1 ='Delivery_location_longitude'
lat2 ='Restaurant_latitude'
long2 ='Restaurant_longitude'

df1['distance'] = df1.loc[:, cols].apply(lambda x: haversine((x[lat1], x[long1]), (x[lat2], x[long2])), axis=1)
avg_distance = df1.loc[:,['City','distance']].groupby('City').mean().reset_index()
fig = go.Figure(data=[go.Pie(labels=avg_distance['City'],values=avg_distance['distance'],pull=[0,0.1,0])])
fig.show()


3. O tempo médio e o desvio padrão de entrega por cidade.

In [30]:
cols = ['City', 'Time_taken(min)',]
df_aux = df1.loc[:,cols].groupby('City').agg({'Time_taken(min)':['mean','std']})

df_aux.columns = ['avg_time','std_time']
df_aux['coef_var'] = df_aux['avg_time']/df_aux['std_time']

df_aux = df_aux.reset_index()

df_aux

,City,avg_time,std_time,coef_var
0,Metropolitian,27.431562,9.126080,3.005843
1,Semi-Urban,50.133333,2.531704,19.802212
2,Urban,23.224289,8.972821,2.588293


In [16]:
cols = ['City', 'Time_taken(min)',]
df_aux = df1.loc[:,cols].groupby('City').agg({'Time_taken(min)':['mean','std']})

df_aux.columns = ['avg_time','std_time']
df_aux['coef_var'] = df_aux['avg_time']/df_aux['std_time']

df_aux = df_aux.reset_index()

fig = go.Figure()
fig.add_trace( go.Bar(
                        name='Control',
                        x=df_aux['City'],
                        y=df_aux['avg_time'],
                        error_y=dict(type='data', array=df_aux['std_time'])))
fig.update_layout(barmode='group')
fig.show()

4. O tempo médio e o desvio padrão de entrega por cidade e tipo de pedido.

In [31]:
cols = ['City', 'Time_taken(min)','Type_of_order']
df_aux = df1.loc[:,cols].groupby(['City','Type_of_order']).agg({'Time_taken(min)':['mean','std']})

df_aux.columns = ['avg_time','std_time']
df_aux['coef_var'] = df_aux['avg_time']/df_aux['std_time']
df_aux = df_aux.reset_index()

df_aux

,City,Type_of_order,avg_time,std_time,coef_var
0,Metropolitian,Buffet,27.273448,9.266798,2.943136
1,Metropolitian,Drinks,27.222078,8.988291,3.028616
2,Metropolitian,Meal,27.276675,9.044081,3.015970
3,Metropolitian,Snack,27.957254,9.205789,3.036921
4,Semi-Urban,Buffet,51.000000,2.645751,19.276188
5,Semi-Urban,Drinks,53.000000,0.000000,inf
6,Semi-Urban,Meal,50.000000,4.000000,12.500000
7,Semi-Urban,Snack,49.000000,1.632993,30.006249
8,Urban,Buffet,22.923077,9.152668,2.504524
9,Urban,Drinks,24.212670,9.071184,2.669185


5. O tempo médio e o desvio padrão de entrega por cidade e tipo de tráfego.

In [18]:
cols = ['City', 'Time_taken(min)','Road_traffic_density']
df_aux = df1.loc[:,cols].groupby(['City','Road_traffic_density']).agg({'Time_taken(min)':['mean','std']})

df_aux.columns = ['avg_time','std_time']
df_aux['coef_var'] = df_aux['avg_time']/df_aux['std_time']
df_aux = df_aux.reset_index()

df_aux

,City,Road_traffic_density,avg_time,std_time,coef_var
0,Metropolitian,High,28.338762,8.036024,3.526466
1,Metropolitian,Jam,32.014300,9.542811,3.354808
2,Metropolitian,Low,22.425265,6.934571,3.233836
3,Metropolitian,Medium,27.976982,8.225346,3.401314
4,Semi-Urban,High,49.000000,NaN,NaN
5,Semi-Urban,Jam,50.909091,2.427120,20.975107
6,Semi-Urban,Medium,47.666667,1.527525,31.205158
7,Urban,High,23.340206,7.847354,2.974277
8,Urban,Jam,28.233463,10.260283,2.751724
9,Urban,Low,19.487324,6.329175,3.078968


In [20]:
cols = ['City', 'Time_taken(min)','Road_traffic_density']
df_aux = df1.loc[:,cols].groupby(['City','Road_traffic_density']).agg({'Time_taken(min)':['mean','std']})

df_aux.columns = ['avg_time','std_time']
df_aux['coef_var'] = df_aux['avg_time']/df_aux['std_time']
df_aux = df_aux.reset_index()

fig = px.sunburst(df_aux,path=['City','Road_traffic_density'],values='avg_time',
                  color='std_time', color_continuous_scale='bluered',
                  color_continuous_midpoint=np.average(df_aux['std_time']))
fig.show()

6. O tempo médio de entrega durantes os Festivais.

In [33]:
cols = ['Time_taken(min)','Festival']
df_aux = df1.loc[:,cols].groupby('Festival').agg({'Time_taken(min)':['mean','std']})

df_aux.columns = ['avg_time','std_time']
df_aux['coef_var'] = df_aux['avg_time']/df_aux['std_time']
df_aux = df_aux.reset_index()

linhas_selecionadas = df_aux['Festival'] == 'Yes'

df_aux = df_aux.loc[linhas_selcionadas, :]

df_aux

,Festival,avg_time,std_time,coef_var
0,No,26.147782,9.018586,2.899322
1,Yes,44.955056,3.999745,11.239482


In [11]:
cols = ['Time_taken(min)','Festival']
df_aux = df1.loc[:,cols].groupby('Festival').agg({'Time_taken(min)':['mean']})

df_aux.columns = ['avg_time']

df_aux = df_aux.reset_index()

df_mean = df_aux.loc[df_aux['Festival'] == 'Yes', 'avg_time']

df_mean

1    44.955056
Name: avg_time, dtype: float64